In [4]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [5]:
cd 'drive'/'MyDrive'/'Dacon'/'LDI'

/content/drive/MyDrive/Dacon/LDI


In [6]:
test_err_data = pd.read_csv('./dataset/test_err_data.csv',thousands = ',')
test_quality_data = pd.read_csv('./dataset/test_quality_data.csv', thousands = ',')

In [7]:
df_tqd = test_quality_data.drop_duplicates()
df_ted = test_err_data.drop_duplicates()

In [8]:
from sklearn.preprocessing import LabelEncoder
encode_index = [1,5,8,9,23,25,38,41] #
df_ted_encoded = pd.DataFrame(columns=["user_id","time","model_nm","fwver","errtype","errcode"])
for i in encode_index:   # Label encoding of errtype
    temp = df_ted[df_ted.errtype == i]
    le = LabelEncoder().fit_transform(temp["errcode"].astype(str))
    temp["errcode"] = le
    df_ted_encoded = pd.concat([df_ted_encoded,temp])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [9]:
df_ted_encoded["model_nm"] = LabelEncoder().fit_transform(df_ted_encoded["model_nm"].astype(str)) # Encoding by model number

In [10]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778','04.82.1684',  # Encoding ted by fwversion
                 '04.33.1185','03.11.1167','04.16.3571','04.73.2237','05.66.3237']
lst_fwv = list(df_ted_encoded['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_ted_encoded['fwver'] = lst_fwv
df_ted_encoded["fwver"] = LabelEncoder().fit_transform(df_ted_encoded["fwver"].astype(str))
df_ted_encoded

,user_id,time,model_nm,fwver,errtype,errcode
5227,30005,20201107095531,4,0,1,0
5248,30005,20201108041628,4,0,1,0
5261,30005,20201109025047,4,0,1,0
5291,30005,20201110024845,4,0,1,0
5322,30005,20201111041705,4,0,1,0
...,...,...,...,...,...,...
16528591,44997,20201127114821,0,4,41,0
16529103,44997,20201127212736,0,4,41,0
16530062,44997,20201128124346,0,4,41,0
16530825,44997,20201129153804,0,4,41,0


In [11]:
encodedfwvlst = ['04.16.3553','04.22.1750','04.33.1261','05.15.2138','04.22.1778','04.82.1684', # Encoding tqd by fwversion
                 '04.33.1185','03.11.1167','04.16.3571','04.73.2237','05.66.3237']
lst_fwv = list(df_tqd['fwver'])
for i in range(len(lst_fwv)):
    if lst_fwv[i] not in encodedfwvlst:
        lst_fwv[i] = 'nope'
df_tqd['fwver'] = lst_fwv
print(df_tqd['fwver'].unique())
df_tqd["fwver"] = LabelEncoder().fit_transform(df_tqd["fwver"].astype(str))

['04.33.1261' '05.15.2138' '04.22.1750' '04.22.1778' '04.16.3553' 'nope'
 '03.11.1167' '04.33.1185' '04.82.1684' '04.16.3571' '04.73.2237'
 '05.66.3237']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [12]:
lst_column = ["user_id", "model_nm", "fwver"]  # Make a output datafarme

for i in range(1,43):
    if i == 29:
        continue
    lst_column.append('errtype_' + str(i))
for i in range(0,13):
    lst_column.append('quality_' + str(i))

df_x = pd.DataFrame(columns = lst_column)
df_x

,user_id,model_nm,fwver,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12


In [14]:
user = list(df_ted['user_id'].unique())
i=1
for user_id in user:
    print(str(i),"/",str(len(user)),"finished.")
    i+=1
    df_check_id_ted = df_ted_encoded[df_ted_encoded.user_id == user_id]
    df_check_id_tqd = df_tqd[df_tqd.user_id == user_id]
    fwvlst = list(df_check_id_ted['fwver'].unique())
    for fwv in fwvlst:
        df_check_fwv_ted = df_check_id_ted[df_check_id_ted.fwver == fwv]
        df_check_fwv_tqd = df_check_id_tqd[df_check_id_tqd.fwver == fwv]
        df_check_fwv_tqd = df_check_fwv_tqd.drop('time', axis = 1)
        df_mean_quality_tqd = df_check_fwv_tqd.mean()
        df_mean_quality_tqd = df_mean_quality_tqd.fillna(0)
        mnlst = list(df_check_fwv_ted['model_nm'].unique())
        for mn in mnlst:
            df_check_mn_ted = df_check_fwv_ted[df_check_fwv_ted.model_nm == mn]
            df_check_mn_ted = df_check_mn_ted.drop('time', axis = 1)
            errtypelst = list(df_check_mn_ted['errtype'].unique())
            errtype_val = {}
            for errtype in errtypelst:
                df_check_ert_ted = df_check_mn_ted[df_check_mn_ted.errtype == errtype]
                df_mean_erc_ted = df_check_ert_ted['errcode'].mean()
                errtype_val[errtype] = df_mean_erc_ted
            result_row = [user_id,mn,fwv]
            e_v_keys = errtype_val.keys()
            for j in range(1,43):
                if j == 29:
                    continue
                if j in e_v_keys:
                    result_row.append(errtype_val[j])
                else:
                    result_row.append(0)
            for j in range(2,15):
                result_row.append(df_mean_quality_tqd[j])
            df_x = df_x.append(pd.Series(result_row,index = df_x.columns),ignore_index=True)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
9999 / 14998 finished.
10000 / 14998 finished.
10001 / 14998 finished.
10002 / 14998 finished.
10003 / 14998 finished.
10004 / 14998 finished.
10005 / 14998 finished.
10006 / 14998 finished.
10007 / 14998 finished.
10008 / 14998 finished.
10009 / 14998 finished.
10010 / 14998 finished.
10011 / 14998 finished.
10012 / 14998 finished.
10013 / 14998 finished.
10014 / 14998 finished.
10015 / 14998 finished.
10016 / 14998 finished.
10017 / 14998 finished.
10018 / 14998 finished.
10019 / 14998 finished.
10020 / 14998 finished.
10021 / 14998 finished.
10022 / 14998 finished.
10023 / 14998 finished.
10024 / 14998 finished.
10025 / 14998 finished.
10026 / 14998 finished.
10027 / 14998 finished.
10028 / 14998 finished.
10029 / 14998 finished.
10030 / 14998 finished.
10031 / 14998 finished.
10032 / 14998 finished.
10033 / 14998 finished.
10034 / 14998 finished.
10035 / 14998 finished.
10036 / 14998 finished.
10037 / 14998 finished.
10038 / 14998 finished.
10039

In [15]:
import pickle
with open('x_test.txt', 'wb') as f:
    pickle.dump(df_x, f)